In [72]:
import sys, os
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image
import pickle

In [19]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) #numpy로 저장된 이미지를 PIL용 데이터 객체로 변환
                                             #uint8 : unsigned integer : 양수값만 표현
    pil_img.show()

In [11]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False) #flatten : 1차원 numpy 배열로

In [14]:
img = x_train[0]
label = t_train[0]
print(label) #5

5


In [15]:
print(img.shape) # (784)
img = img.reshape(28, 28) # 원래 이미지의 모영으로 변형
print(img.shape)

(784,)
(28, 28)


In [20]:
img_show(img)  #MNIST 이미지 출력

In [64]:
os.getcwd() #현재 작업위치 확인

os.chdir('ch03') #작업위치 변경

os.listdir() #작업 디렉토리 파일 확인

In [81]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c) ## 오버플로 대비
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [77]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

In [28]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False) 
    return x_test, t_test

In [73]:
def init_network(): 
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        
    return network #학습된 가중치 매개변수 가져오기

In [79]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) +b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [82]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i] :
        accuracy_cnt += 1
        
print("Accuracy:" + str(float(accuracy_cnt) /len(x)))

Accuracy:0.9352


In [87]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3'] #학습된 매개변수 W1, W2, W3

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)
## 784가 마지막에는 원소 10개인 1차원 배열이 출력됨.`

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [98]:
## 배치처리 구현
x, t = get_data()
network = init_network()

batch_size = 100 #한번에 들어가는 데이터 양 
                 # ex) size = 10,000, batch_size = 100이면 한 번에 100개 데이터씩 묶어서 예측 진행
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch) # x_batch로 불러온 매개변수를 이용하여 y 예측
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])  #정확도 합 계산
    
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
